In [399]:
import numpy as np
import pandas as pd

In [400]:
dataSet=pd.read_csv("../input_data/apparel-trainval.csv")

In [927]:
trainingData=dataSet.sample(frac=0.8)
validationData=dataSet.drop(trainingData.index)

trainingLabels=np.array(trainingData['label'])
validationLabels=np.array(validationData['label'])

trainingData=np.array(trainingData.drop('label',axis=1))
validationData=np.array(validationData.drop('label',axis=1))

In [948]:
class Layer():
    def __init__(self,currentSize):
        self.layerSize=currentSize
        self.Weights=None
        self.Bias=None
        self.Z=None
        self.Activations=None
        self.ActFunction=None
        self.dActFunction=None

class NeuralNetwork():
    
    def __init__(self,nLayers,layerSizes,hLayerActFunction):
        
        self.nLayers=nLayers
        self.layers=[]
        for x in range(self.nLayers):
            self.layers.append(Layer(layerSizes[x]))
        
        for x in range(1,self.nLayers):
            self.layers[x].Weights=np.random.randn(layerSizes[x],layerSizes[x-1])*0.01
            self.layers[x].Bias=np.zeros((layerSizes[x],1))
            self.layers[x].ActFunction=getattr(self,hLayerActFunction)
            self.layers[x].dActFunction=getattr(self,"der"+hLayerActFunction)
        
        self.layers[-1].ActFunction=getattr(self,"SIGMOID")
        
    def forwardProp(self,X): # making the prediction on current weights
        self.layers[0].Activations=X.T
        for i in range(1,self.nLayers-1):
            self.layers[i].Z=np.dot(self.layers[i].Weights,self.layers[i-1].Activations)+self.layers[i].Bias
            self.layers[i].Activations=self.layers[i].ActFunction(self.layers[i].Z)
        # for last layer
        self.layers[-1].Z=np.dot(self.layers[-1].Weights,self.layers[-2].Activations)+self.layers[-1].Bias
        self.layers[-1].Activations=self.layers[-1].ActFunction(self.layers[-1].Z)
        
    def cost(self,Y):
        X=self.layers[-1].Activations
        Y=self.makeVector(Y)
        return -(np.sum(Y*np.log(X)+(1-Y)*(np.log(1-X))))/len(Y)
    
    def backProp(self,X,Y): # back propogating the error
        
        self.layers[-1].dZ=self.layers[-1].Activations-self.makeVector(Y)
        self.layers[-1].dW=np.dot(self.layers[-1].dZ,self.layers[-2].Activations.T)
        self.layers[-1].dB=(np.sum(self.layers[-1].dZ,axis=1,keepdims=True))/len(Y)
        
        for i in range(self.nLayers-2,0,-1):
            self.layers[i].dZ=np.dot(self.layers[i+1].Weights.T,self.layers[i+1].dZ)*(self.layers[i].dActFunction(self.layers[i].Z))
            self.layers[i].dW=np.dot(self.layers[i].dZ,self.layers[i-1].Activations.T)/len(Y)
            self.layers[i].dB=(np.sum(self.layers[i].dZ,axis=1,keepdims=True))/len(Y)
    
        for i in range(1,self.nLayers):
            self.layers[i].Weights-=0.001*(self.layers[i].dW)
            self.layers[i].Bias-=0.001*(self.layers[i].dB)
    
    def derSIGMOID(self,z): # derivative of sigmoid function
        s=self.SIGMOID(z)
        return s*(1-s)
    
    def SOFTMAX(self,x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)
    
    def TANH(self,x):
        e_2x=np.exp(2*x)
        return (e_2x-1)/(e_2x+1)
    
    def derTANH(self,x):
        s=self.TANH(x)
        return 1-s**2
    
    def RELU(self,z):
        return np.maximum(z, 0)
    
    def derRELU(self,z):
        return np.greater(z, 0).astype(int)
    
    def SIGMOID(self,z):
        return 1/(1+np.exp(-z))
    
    def makeVector(self,Y):
        labels=[]
        for i in Y:
            arr=np.zeros(10)
            arr[i]=1
            labels.append(arr)
        return np.array(labels).T
    
    def fit(self,X,Y,batchSize): # gradient Descent
        for i in range(40):
            for j in range(0,len(X),batchSize):
                tempX=X[j:j+batchSize,:]
                tempY=Y[j:j+batchSize]
                self.forwardProp(tempX)
                self.backProp(tempX,tempY)
            print("iteration: ",i)
    def predict(self,X):
        self.forwardProp(X)
        predictions=self.SOFTMAX(self.layers[-1].Activations).T
        return np.array([np.argmax(item) for item in predictions])
    
    def checkAccuracy(self,A,B):
        predictions=self.predict(A)
        c=0
        for a,b in zip(predictions,B):
            if a==b:
                c+=1
        return c*100/len(B)

In [949]:
for func in ["RELU","SIGMOID"]:
    model=NeuralNetwork(4,[trainingData.shape[1],100,100,10],func)
    print("Training with "+func+"...")
    model.fit(trainingData,trainingLabels,100)
    print("Saving Weights & Bias...")
    for i in range(1,model.nLayers):
        np.save("Weight_"+func+"_"+str(i)+".npy",model.layers[i].Weights)
        np.save("Bias_"+func+"_"+str(i)+".npy",model.layers[i].Bias)

Training with RELU...
iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
iteration:  9
iteration:  10
iteration:  11
iteration:  12
iteration:  13
iteration:  14
iteration:  15
iteration:  16
iteration:  17
iteration:  18
iteration:  19
iteration:  20
iteration:  21
iteration:  22
iteration:  23
iteration:  24
iteration:  25
iteration:  26
iteration:  27
iteration:  28
iteration:  29
iteration:  30
iteration:  31
iteration:  32
iteration:  33
iteration:  34
iteration:  35
iteration:  36
iteration:  37
iteration:  38
iteration:  39
Saving Weights & Bias...
Training with SIGMOID...
iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
iteration:  9
iteration:  10
iteration:  11
iteration:  12
iteration:  13
iteration:  14
iteration:  15
iteration:  16
iteration:  17
iteration:  18
iteration:  19
iteration:  20
iteration:  21
iteration:  22
ite

In [951]:
for func in ["RELU","SIGMOID"]:
    model=NeuralNetwork(4,[trainingData.shape[1],100,100,10],func)
    for i in range(1,model.nLayers):
        model.layers[i].Weights=np.load("Weight_"+func+"_"+str(i)+".npy")
        model.layers[i].Bias=np.load("Bias_"+func+"_"+str(i)+".npy")
    print("Accuracy with "+func+" is: ",model.checkAccuracy(validationData,validationLabels))

Accuracy with RELU is:  88.01666666666667
Accuracy with SIGMOID is:  83.55


In [957]:
testingData=np.array(pd.read_csv("../input_data/apparel-test.csv"))

In [970]:
model=NeuralNetwork(4,[testingData.shape[1],100,100,10],"RELU")
for i in range(1,model.nLayers):
    model.layers[i].Weights=np.load("Weight_"+"RELU"+"_"+str(i)+".npy")
    model.layers[i].Bias=np.load("Bias_"+"RELU"+"_"+str(i)+".npy")
predictions=model.predict(testingData)

In [971]:
with open("../output_data/2018202001_prediction.csv", "wb") as f:
    f.write(b'pLabel\n')
    np.savetxt(f, predictions.astype(int), fmt='%i', delimiter=",")